In [1]:
import pandas as pd
import io

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)     
pd.set_option('display.width', 0)  

# ====================  1 ====================
print("--- 1: การโหลดและตรวจสอบข้อมูลเบื้องต้น ---")
print()

# โหลดข้อมูล
df = pd.read_csv('student_data.csv', encoding='utf-8-sig')
df.columns = df.columns.str.strip()             
df.columns = df.columns.str.replace('\ufeff', '') 


print("แสดงข้อมูล 10 แถวแรก:")
print(df.head(10))
print()

print("ตรวจสอบชนิดข้อมูลและค่าว่างด้วย .info():")
print(df.info())
print()

print("คำตอบคำถามข้อ 1:")
print("คอลัมน์ที่มีข้อมูลขาดหายคือ MidtermScore (2 ค่า) และ FinalScore (1 ค่า)")
print("ชนิดข้อมูลเป็น float64 เนื่องจาก Pandas จะแปลงคอลัมน์ที่มีค่าว่างเป็น float โดยอัตโนมัติ")
print("-" * 50)
print()

# ====================  2 ====================
print("--- : การจัดการข้อมูลที่ขาดหาย ---")


midterm_median = df['MidtermScore'].median()
final_median = df['FinalScore'].median()


df['MidtermScore'] = df['MidtermScore'].fillna(df['MidtermScore'].median())
df['FinalScore'] = df['FinalScore'].fillna(df['FinalScore'].median())

print("เติมค่าที่ขาดหายด้วย median เรียบร้อยแล้ว")
print("ตรวจสอบข้อมูลขาดหายอีกครั้ง:")
print(df.isnull().sum())
print("-" * 50)
print()

# ====================  3 ====================
print("---  3: การแปลงชนิดข้อมูล ---")

print(f"ชนิดข้อมูลของ ProjectScore ก่อนแปลง: {df['ProjectScore'].dtype}")
df['ProjectScore'] = df['ProjectScore'].astype(int)
print(f"ชนิดข้อมูลของ ProjectScore หลังแปลง: {df['ProjectScore'].dtype}")
print("แปลงชนิดข้อมูลเรียบร้อยแล้ว")
print("-" * 50, "\n")

# ====================  4 ====================
print("---  4: การสร้างคอลัมน์ FullName ---")

df['FullName'] = df['FirstName'] + ' ' + df['LastName']
print("สร้างคอลัมน์ FullName เรียบร้อยแล้ว:")
print(df[['FirstName', 'LastName', 'FullName']].head())
print("-" * 50, "\n")

# ====================  5 ====================
print("---  5: การคำนวณคะแนนรวม ---")

df['TotalScore'] = df['MidtermScore'] + df['FinalScore'] + df['ProjectScore']
print("สร้างคอลัมน์ TotalScore เรียบร้อยแล้ว:")
print(df[['FullName', 'MidtermScore', 'FinalScore', 'ProjectScore', 'TotalScore']].head())
print("-" * 50, "\n")

# ====================  6 ====================
print("---  6: การคำนวณและตัดเกรด ---")

def calculate_grade(total_score):
    if total_score >= 210:
        return 'A'
    elif total_score >= 187.5:
        return 'B+'
    elif total_score >= 175:
        return 'B'
    elif total_score >= 162.5:
        return 'C+'
    elif total_score >= 150:
        return 'C'
    elif total_score >= 137.5:
        return 'D+'
    elif total_score >= 125:
        return 'D'
    else:
        return 'F'

df['FinalGrade'] = df['TotalScore'].apply(calculate_grade)
print("สร้างคอลัมน์ FinalGrade เรียบร้อยแล้ว:")
print(df[['FullName', 'TotalScore', 'FinalGrade']].head())
print("-" * 50, "\n")

# ====================  7 ====================
print("--- 7: การคัดกรองข้อมูล ---")


df['Attendance'] = pd.to_numeric(df['Attendance'], errors='coerce')
df['Attendance'] = df['Attendance'].fillna(df['Attendance'].median())


comp_eng_grade_a = df[(df['Major'] == 'Computer Engineering') & (df['FinalGrade'] == 'A')]
print("นักศึกษาวิศวกรรมคอมพิวเตอร์ที่ได้เกรด 'A':")
print(comp_eng_grade_a[['FullName', 'Major', 'TotalScore', 'FinalGrade']], "\n")


low_attendance = df[df['Attendance'] < 70]
print("นักศึกษาที่มีการเข้าเรียนน้อยกว่า 70%:")
print(low_attendance[['FullName', 'Major', 'Attendance']])
print("-" * 50, "\n")

# ====================  8 ====================
print("--- 8: การเรียงลำดับข้อมูล ---")

top_5_students = df.sort_values('TotalScore', ascending=False).head(5)
print("5 อันดับแรกของนักศึกษาที่มีคะแนนรวมสูงสุด:")
print(top_5_students[['FullName', 'Major', 'TotalScore', 'FinalGrade']])
print("-" * 50, "\n")

# ====================  9 ====================
print("--- 9: การจัดกลุ่มและสรุปผลข้อมูล ---")

major_summary = df.groupby('Major').agg({
    'TotalScore': 'mean',
    'FinalScore': 'mean'
}).round(2)

major_summary.columns = ['Average_TotalScore', 'Average_FinalScore']
print("คะแนนเฉลี่ยของแต่ละสาขาวิชา:")
print(major_summary, "\n")

highest_major = major_summary['Average_TotalScore'].idxmax()
print("คำตอบคำถามข้อ 9:")
print(f"สาขาวิชาที่มีคะแนนรวมเฉลี่ยสูงสุดคือ: {highest_major}")
print("-" * 50, "\n")

# ==================== ข้อที่ 10 ====================
print("---  10: การเตรียมและการบันทึกรายงานสรุป ---")

report_df = df[['StudentID', 'FullName', 'Major', 'TotalScore', 'FinalGrade']]
print("ตัวอย่างข้อมูลใน report_df:")
print(report_df.head(), "\n")

report_df.to_csv('student_grade_report.csv', index=False)
print("ไฟล์ 'student_grade_report.csv' ถูกสร้างเรียบร้อยแล้ว")
print("-" * 50)

--- 1: การโหลดและตรวจสอบข้อมูลเบื้องต้น ---

แสดงข้อมูล 10 แถวแรก:
   StudentID FirstName   LastName                   Major  MidtermScore  \
0    6401001   Somchai     Jaidee    Computer Engineering          85.0   
1    6401002    Somsri     Rakdee    Computer Engineering          76.0   
2    6401003      Mana      Petch       Civil Engineering          65.0   
3    6401004    Chujai      Klaew  Electrical Engineering          92.0   
4    6401005      Piti     Yindee    Computer Engineering          88.0   
5    6401006     Malee     Boonma       Civil Engineering           NaN   
6    6401007     Weera    Chokdee  Electrical Engineering          78.0   
7    6401008     Jinda     Suksai    Computer Engineering          95.0   
8    6401009      Arun  Rungrueng  Industrial Engineering          55.0   
9    6401010     Naree    Ngamdee       Civil Engineering          72.0   

   FinalScore  ProjectScore  Attendance  
0        90.0          45.0          95  
1         NaN          